In [1]:
# ch4-2.py
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

In [4]:
# 데이터로드 (ch4-2(붓꽃데이터).csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData_org = pd.read_csv('./NHIS_OPEN_GJ_2017.csv', encoding='CP949', engine='python')

In [5]:
# 원본데이터의 튜플 수와 컬럼수 출력
rawData_org.shape

(1000000, 34)

In [6]:
# 의사결정트리에 사용할 속성리스트
feature_columns_to_use = ['성별코드', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레', '시력(좌)', '시력(우)',
                    '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '트리글리세라이드', 'HDL콜레스테롤',
                    'LDL콜레스테롤', '요단백', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마지티피', '흡연상태', '음주여부']

rawData = rawData_org[feature_columns_to_use]

In [7]:
# 속성값이 NULL인 튜플 제외
rawData = rawData.dropna()

# 인덱스 재설정
rawData.reset_index(inplace=True, drop=True)

# 원본데이터의 튜플 수와 컬럼수 출력
rawData.shape

(990910, 22)

In [8]:
import random as rd  # 샘플링을 위한 random 패키지 임포트

# 1) 샘플링을 통한 수량 축소
# 원본데이터 중 50,000개를 튜플만 샘플링
# len() : 데이터프레임의 크기를 구하는 함수
sample_idx = rd.sample(range(0, len(rawData)), 50000)

# 인덱스 값 정렬(오름차순)
sample_idx.sort()

sample_idx[:10]

[5, 6, 8, 14, 28, 116, 125, 142, 159, 163]

In [9]:
# 샘플링 된 인덱스로 구성된 샘플 데이터프레임 생성
rawData_sample = rawData.loc[sample_idx]

rawData_sample.reset_index(inplace=True, drop=True) #  인덱스 재설정

# 첫 10개의 행만 출력
rawData_sample.head(10)

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부
0,1,8,170.0,70.0,84.8,1.2,1.0,1.0,1.0,115.0,...,139.0,46.0,136.0,1.0,0.9,25.0,30.0,33.0,2.0,1.0
1,1,8,175.0,110.0,111.5,1.0,1.0,1.0,1.0,130.0,...,126.0,58.0,174.0,1.0,1.0,28.0,51.0,45.0,3.0,1.0
2,1,8,170.0,80.0,93.0,0.7,0.9,1.0,1.0,115.0,...,185.0,32.0,89.0,1.0,1.0,31.0,29.0,51.0,2.0,0.0
3,2,10,160.0,70.0,87.0,0.8,0.8,1.0,1.0,129.0,...,173.0,56.0,164.0,1.0,0.7,16.0,12.0,12.0,1.0,0.0
4,1,7,175.0,75.0,76.0,0.3,0.4,1.0,1.0,131.0,...,372.0,37.0,116.0,1.0,0.8,20.0,27.0,30.0,3.0,0.0
5,1,9,175.0,85.0,93.0,1.5,1.5,1.0,1.0,128.0,...,268.0,54.0,141.0,1.0,1.2,24.0,28.0,28.0,2.0,1.0
6,2,6,160.0,55.0,70.0,1.2,0.9,1.0,1.0,110.0,...,52.0,54.0,125.0,1.0,0.8,17.0,13.0,14.0,1.0,1.0
7,1,10,160.0,60.0,81.0,1.2,1.0,1.0,1.0,124.0,...,111.0,64.0,143.0,1.0,0.9,36.0,38.0,46.0,2.0,0.0
8,1,13,175.0,80.0,95.0,1.5,1.2,1.0,1.0,129.0,...,117.0,62.0,123.0,1.0,1.1,25.0,30.0,21.0,2.0,0.0
9,2,15,140.0,55.0,77.0,0.5,0.8,1.0,1.0,130.0,...,92.0,52.0,183.0,1.0,1.1,25.0,15.0,15.0,1.0,0.0


In [10]:
rawData_sample.shape

(50000, 22)

In [11]:
# nonnumeric 속성을 categrical 데이터로 변환
nonnumeric_columns = ['성별코드','음주여부']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in nonnumeric_columns:
    rawData_sample[feature] = le.fit_transform(rawData_sample[feature])   

In [12]:
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
X = rawData_sample.loc[:, feature_columns_to_use[:-1]]  
y = rawData_sample.loc[:, '음주여부']  # 분류 클래스(class)

from sklearn.model_selection import train_test_split  # 분석모형 선택에 관련된 모듈
    
# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [13]:
# RMSLE 계산하는 사용자정의 함수
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values) :
    # 넘파이로 배열 형태로 변환
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제값에 1을 더하고 로그를 씌움
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 함
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균값 구함
    mean_difference = difference.mean()
    
    # 다시 루트를 씌움
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [15]:
# Scikit-Learn 패키지 : 머신 러닝 교육 및 실무를 위한 패키지로 샘플 데이터셋,
# 다양한 기계학습 기법에 대한 함수 등을 포함하고 있음
import xgboost as xgb # xgboost 기법에 관련된 모듈

gbm = xgb.XGBClassifier(max_depth=4, n_estimators=300, learning_rate=0.05)

gbm

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.05,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=300, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [16]:
# KFold 교차검증
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

# 트레이닝셋에 대한 KFold 교차검증 수
%time score = cross_val_score(gbm, X_train, y_train, cv=k_fold, scoring=rmsle_scorer)
score = score.mean()

# 0에 근접할수록 좋은 데이터
print("Score= {0:.5f}".format(score))

CPU times: user 1min, sys: 235 ms, total: 1min
Wall time: 55 s
Score= 0.35482


In [17]:
# fit() : 트레이닝 데이터셋을 대상으로 의사결정트리 학습 진행
gbm.fit(X_train, y_train)

# tree.predict() 함수를 활용하여 의사결정트리를 대상으로 테스트셋을 예측
y_pred_tr = gbm.predict(X_test)

In [18]:
# 점수 출력
print("Train Set Score1 : {:.2f}".format(gbm.score(X_train, y_train)))
print("Test Set Score1 : {:.2f}".format(gbm.score(X_test, y_test)))

Train Set Score1 : 0.76
Test Set Score1 : 0.74


In [19]:
from sklearn.metrics import accuracy_score  # 분류 정확도(classification accuracy)를 계산하는 모듈

# accuracy_score() 함수를 활용하여 테스트셋의 실제 클래스와 예측된 클래스 간 정확도 측정
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_tr))

Accuracy: 0.74


In [20]:
# 속성(feature) 별 중요도를 저장하는 데이터프레임 생성 
sel_feature = pd.DataFrame({'중요도' : gbm.feature_importances_}, index = feature_columns_to_use[:-1])

# 중요도의 내림차순으로 정렬
sel_feature.sort_values(by='중요도', ascending=False)

,중요도
성별코드,0.350400
흡연상태,0.324082
연령대코드(5세단위),0.088464
신장(5Cm단위),0.052586
감마지티피,0.043161
HDL콜레스테롤,0.025747
(혈청지오티)ALT,0.017034
체중(5Kg 단위),0.011557
이완기혈압,0.009684
트리글리세라이드,0.008615
